In [1]:
import librosa
import numpy as np
import matplotlib
import pandas as pd

In [2]:
### 만약 librosa 버전이 0.8.0 이라면 이 패키지를 임폴트해야함.

import soundfile as sf

In [3]:
print(librosa.__version__)

0.5.1


In [4]:
audio = './0001-1.wav'

In [5]:
audio[2:]

'0001-1.wav'

In [6]:
#1개의 데이터에만 적용
#백색잡음 추가(white noise)

def add_white_noise(data, sr=22050, noise_rate=0.005):
    y, sr = librosa.load(data)
    wn = np.random.randn(len(y))
    y_wn = y + noise_rate*wn
    sf.write('./wn_%s' % (data[2:]), y_wn, sr) 
    print('White Noise 저장 성공')

In [7]:
add_white_noise(audio)

FileNotFoundError: [Errno 2] No such file or directory: '/home/nextgen/Desktop/breath/breathdata_32/0001-1.wav'

In [ ]:
#1개의 데이터에만 적용
#0.8배속(change speed)

def stretch_sound(data, sr=22050, rate=0.8):
    y, sr = librosa.load(data)
    stretch_data = librosa.effects.time_stretch(y, rate)
    sf.write('./cs_%s' % (data[2:]), stretch_data, sr)
    print('stretch_data 저장 성공')

In [ ]:
stretch_sound(audio)

In [ ]:
#1개의 데이터에만 적용
#pitch 변경(steps를 +를 줘야할지, -를 줘야할지 잘.... 주로 -5?)

def pitch(data, sr=22050, n_steps=-5):
    y, sr = librosa.load(data)
    pitch_data = librosa.effects.pitch_shift(y, sr, n_steps)
    sf.write('./cp_%s' % (data[2:]), pitch_data, sr)
    print('stretch_data 저장 성공')

In [ ]:
pitch(audio)

In [ ]:
#1개의 데이터에만 적용
#Time Shifting

def shifting_sound(data, sr=22050, roll_rate=0.1):
    y, sr = librosa.load(data)
    data_roll = np.roll(y, int(sr*roll_rate))
    sf.write('./ts_%s' % (data[2:]), data_roll, sr)
    print('rolling_sound 저장 성공')

In [ ]:
shifting_sound(audio)

In [ ]:
#1개의 데이터에만 적용
#Reverse sound

def reverse_sound(data, sr=22050):
    y, sr = librosa.load(data)
    reverse_y = np.array([y[len(y)-1-i] for i in range(len(y))])
    sf.write('./rs_%s' % (data[2:]), reverse_y, sr)
    print('reverse_sound 저장 성공')

In [ ]:
reverse_sound(audio)

In [ ]:
#1개의 데이터에만 적용
#minus(위아래 높낮이 바꾸기(뒤집기))

def minus_sound(data, sr=22050):
    y, sr = librosa.load(data)
    temp_y = (-1)*y
    sf.write('./minus_%s' % (data[2:]), temp_y, sr)
    print('rever_높낮이 저장 성공')

In [ ]:
minus_sound(audio)

In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
train.head()

In [ ]:
len(train)

In [ ]:
train['relative_path'] = '/' + train['filename'].astype(str)
train = train[['relative_path', 'class']]
train.head()

In [ ]:
print(train.shape)

In [ ]:
for i in range(len(train)):
    audio = '.'+train.iloc[i,0]
    add_white_noise(audio,sr=32000)
    stretch_sound(audio,sr=32000)
    pitch(audio,sr=32000)
    shifting_sound(audio,sr=32000)
    reverse_sound(audio,sr=32000)
    minus_sound(audio,sr=32000)

In [ ]:
### 기존 train/val 데이터의 7배 증강
### 6가지 방법 중 뭔가 느낌이 증강을 한다고 과연 좋을까란 의문이 듬. 속도같은경우도 배속을 빠르게보단 느리게가 좋을 것같아서 0.8로했고,
### reverse sound에 대해서도 의문, 잡음이 많이 없는데 백색잡음이 좋을지에 대해서도, 주파수 위아래를 뒤집는 것도 좋을지에 대해 의문..
### test 셋을 20% 떼내고 나머지 데이터에 대해 csv 경로 파일에서 for문을 통해 모든 데이터에 적용가능.
### 그 후 증강된 데이터셋에 대해 새로 csv 경로 및 레이블 파일 작성..